## Run the Self-Organizing Map
___

First, load the data

In [1]:
from astropy.io import fits
from astropy.table import Table

import numpy as np

In [ ]:
#Load in data produced in primary_targeting_region.ipynb
cut_data_file = '/data2/lsajkov/mpdg/data_products/GAMA/GAMA_primtarg_snr100_lms6_12_25jun2024.fits'

with fits.open(cut_data_file) as cat:
    GAMA_vect_data = Table(cat[1].data)

GAMA_vect_data.add_column(GAMA_vect_data['r_mag_err'], index = 4, name = 'surf_bright_r_err')

In [ ]:
#Select the needed data
input_data = GAMA_vect_data['r_mag', 'gr_color', 'surf_bright_r']
input_stds = GAMA_vect_data['r_mag_err', 'gr_color_err', 'surf_bright_r_err']

input_labels = GAMA_vect_data['log_stellar_mass', 'redshift']

tuple_labels = input_labels.as_array()
list_labels = [list(values) for values in tuple_labels]
input_labels = np.array(list_labels)

In [ ]:
#Normalize the data to mean 0, unit variance
normalization_params = {}

for variable in input_data.keys():
    normalization_params[variable] = {}

    normalization_params[variable]['mean'] = np.mean(GAMA_vect_data[variable])
    normalization_params[variable]['std']  = np.std(GAMA_vect_data[variable])


normalized_input_data = input_data.copy()
for variable in input_data.keys():
    
    normalized_input_data[variable] -= normalization_params[variable]['mean']
    normalized_input_data[variable] /= normalization_params[variable]['std']

___

Initialize the SOM

In [ ]:
from mpdg_som import SelfOrganizingMap

In [ ]:
#Set parameters
name = 'mass_profile_dwarf_galaxies' #name of the SOM

mapsize   = [16, 16] #size of the map. pass as a list of dimensions OR as an integer (also pass number of dimensions)
dimension = None

initialization         = 'pca' #random or pca (principal component analysis)
termination            = 'maximum_steps' #when to stop learning. maximum_steps = stop when maximum_steps have elapsed. error_thresh = stop when the error is below this threshold.
learning_rate_function = 'power_law' #which learning rate function to use. currently implemented: power_law
neighborhood_function  = 'gaussian' #which neighborhood function to use. currently implemented: gaussian
error_estimator        = None #which error estimation function to use. currently implemented: max_misalignment

learning_rate = 0.5 #used to adjust the learning rate function
maximum_steps = 100 #used to adjust the learning rate and neighborhood functions
error_thresh  = None #used to stop the SOM if termination = 'error thresh'

In [ ]:
#Declare the SOM
SOM = SelfOrganizingMap(
    name                   = name,
    mapsize                = mapsize,
    initialization         = initialization,
    termination            = termination,
    learning_rate_function = learning_rate_function,
    neighborhood_function  = neighborhood_function,
    learning_rate          = learning_rate,
    maximum_steps          = maximum_steps
)

In [ ]:
#Load the data & its stds. Check the statistics to make sure it's normalized.
data_cut = 1000 #use up to this much of the data
SOM.load_data(normalized_input_data[:data_cut],
              variable_names = ['r_mag', 'gr_color', 'surf_bright_r'])
SOM.load_standard_deviations(input_stds[:data_cut])
SOM.data_statistics()

In [ ]:
#Initialize the SOM
SOM.build_SOM()

In [ ]:
#Visualize SOM before training
SOM.show_map()

In [ ]:
#Train the som
SOM.train()

In [ ]:
#Visualize the SOM after training
SOM.show_map()

In [ ]:
#Label the cells in the map with external parameters. Visualize the labeled map.
SOM.label_map(input_labels[:data_cut],
              parameter_names = ['log_mstar', 'redshift'])
SOM.show_map(show_labeled = True)